In [1]:
import os
import sys
current_directory = os.getcwd()
if not os.path.exists(current_directory + '\\experimental_data'):
    os.makedirs(current_directory + '\\experimental_data')
    os.makedirs(current_directory + '\\experimental_data\\plots')
    os.makedirs(current_directory + '\\experimental_data\\results')
sys.path.append(current_directory.replace('experiments', ''))
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.set_loglevel('WARNING')
from run import execute_simulation, create_experiment_snapshots, execute_ai_model
from mtdnetwork.mtd.completetopologyshuffle import CompleteTopologyShuffle
from mtdnetwork.mtd.ipshuffle import IPShuffle
from mtdnetwork.mtd.hosttopologyshuffle import HostTopologyShuffle
from mtdnetwork.mtd.portshuffle import PortShuffle
from mtdnetwork.mtd.osdiversity import OSDiversity
from mtdnetwork.mtd.servicediversity import ServiceDiversity
from mtdnetwork.mtd.usershuffle import UserShuffle
from mtdnetwork.mtd.osdiversityassignment import OSDiversityAssignment
import logging
import pandas as pd
import numpy as np
from math import pi


logging.basicConfig(format='%(message)s', level=logging.INFO)

In [2]:
create_experiment_snapshots([25, 50, 75, 100])

In [3]:
# Learning Parameters
epsilon = 1.0  # exploration rate

# Simulator Settings
start_time = 0
finish_time = 3000
mtd_interval = 100
total_nodes = 300
new_network = True
features = ["host_compromise_ratio", "exposed_endpoints", "attack_path_exposure",  "overall_asr_avg", "roa", "shortest_path_variability", "risk"]
model = "main_network_final_host_compromise_ratio#exposed_endpoints#attack_path_exposure#overall_asr_avg#roa#shortest_path_variability#risk"
trial = 5


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class RadarPlot:
    def __init__(self, epsilon, start_time, finish_time, mtd_interval, total_nodes, new_network, features, model, trial):
        # Learning Parameters
        self.epsilon = epsilon  # exploration rate

        # Simulator Settings
        self.start_time = start_time
        self.finish_time = finish_time
        self.mtd_interval = mtd_interval
        self.schemes = [ 'mtd_ai', 'simultaneous', 'random', 'alternative']
        self.total_nodes = total_nodes
        self.new_network = new_network
        self.features = features
        self.model = model
        self.trial = trial
        self.normalization_values = self.calculate_normalized_values()
        # print(self.normalization_values)




    def calculate_normalized_values(self):
        # DataFrame to store the results of each trial
        security_metrics = pd.DataFrame(columns=self.features)

        for _ in range(self.trial):
            # Execute the simulation and collect the results
            evaluation = execute_simulation(
                features=features,
                start_time=self.start_time,
                finish_time=self.finish_time,
                scheme="None",
                mtd_interval=self.mtd_interval,
                total_nodes=self.total_nodes,
                new_network=self.new_network
            )
            # Extract the latest security metrics record from the simulation
            security_metrics_trial = evaluation.security_metrics_record.get_record().iloc[-1]
            security_metrics = pd.concat([security_metrics, pd.DataFrame([security_metrics_trial])], ignore_index=True)

        # Calculate median for normalization
        normalized_values = security_metrics.median().to_dict()

        return normalized_values

    
    def compute_metrics(self, scheme,normalization_values=None):
            # DataFrame to store the results of each trial
            security_metrics = pd.DataFrame(columns=self.features)

            for _ in range(self.trial):
                # Execute the AI model and collect the results
                evaluation = execute_ai_model(
                    model=self.model,
                    features=self.features,
                    start_time=self.start_time,
                    finish_time=self.finish_time,
                    mtd_interval=self.mtd_interval,
                    scheme= scheme,
                    total_nodes=self.total_nodes,
                    new_network=self.new_network
                )

                # Extract the latest security metrics record
                security_metrics_trial = evaluation.security_metrics_record.get_record().drop('times', axis=1).iloc[-1]
                security_metrics = pd.concat([security_metrics, pd.DataFrame([security_metrics_trial])], ignore_index=True)

            # Normalize each column if normalization values are provided
            if normalization_values:
                for column, norm_value in normalization_values.items():
                    if column in security_metrics.columns:
                        if norm_value != 0:  # Check if norm_value is not zero to avoid division by zero
                            if column in ["host_compromise_ratio", "exposed_endpoints", "attack_path_exposure", "risk"]:
                                security_metrics[column] = 1 / (security_metrics[column] / norm_value)
                            else:
                                security_metrics[column] = security_metrics[column] / norm_value
                        else:
                            # Handle the case where norm_value is zero
                            # You can either set the result to NaN, zero, or any other placeholder value
                            security_metrics[column] = 1  

            # Calculate median and standard deviation for each metric
            results = {
                'Median': security_metrics.median(),
                'Standard Deviation': security_metrics.std()
            }

            # Convert results to a DataFrame
            results_security_metrics = pd.DataFrame(results).dropna()

            # Extract data for the radar plot
            metrics_values = results_security_metrics['Median'].tolist()
            labels = results_security_metrics.index.tolist()

            return labels, metrics_values, results_security_metrics['Standard Deviation'].tolist()
    

    def plot_radar(self, labels, metrics_values, std_dev_values, scheme):
        # Number of metrics
        num_vars = len(labels)

        # Compute angle for each axis
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        metrics_values += metrics_values[:1]
        std_dev_values += std_dev_values[:1]
        labels += labels[:1]  # Close the circle by repeating the first label

        # Create radar plot
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

        # Plot data
        ax.fill(angles, metrics_values, color='blue', alpha=0.25)
        ax.plot(angles, metrics_values, color='blue', linewidth=2, label='Median')

        # Add error bars
        for i in range(num_vars):
            angle = angles[i]
            value = metrics_values[i]
            error = std_dev_values[i]
            ax.errorbar(angle, value, yerr=error, fmt='o', color='blue', capsize=5, elinewidth=2)

        # Add score labels
        for i in range(num_vars):
            angle = angles[i]
            value = metrics_values[i]
            ax.text(angle, value + 0.05, f'{value:.2f}', horizontalalignment='center', size=10, color='black')

        # Labels
        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])  # Set ticks without the last angle
        ax.set_xticklabels(labels[:-1], rotation=45, ha='right')

        plt.title(f"Radar chart with AI with all 4 single MTD (compare to {scheme}) over {self.trial} trials", size=15, color='blue', y=1.1)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
        plt.show()


    def plot_stacked_radar(self, metrics_data, scheme_names):
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

        # Define colors for each scheme
        colors = ['blue', 'green', 'red', 'orange', 'purple', 'cyan']

        for i, (labels, metrics_values, std_dev_values) in enumerate(metrics_data):
            num_vars = len(labels)
            angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
            angles += angles[:1]  # Complete the circle
            metrics_values += metrics_values[:1]
            std_dev_values += std_dev_values[:1]
            labels += labels[:1]

            ax.fill(angles, metrics_values, color=colors[i % len(colors)], alpha=0.25)
            ax.plot(angles, metrics_values, color=colors[i % len(colors)], linewidth=2, label=scheme_names[i])

            for j in range(num_vars):
                angle = angles[j]
                value = metrics_values[j]
                error = std_dev_values[j]
                ax.errorbar(angle, value, yerr=error, fmt='o', color=colors[i % len(colors)], capsize=5, elinewidth=2)

            for j in range(num_vars):
                angle = angles[j]
                value = metrics_values[j]
                ax.text(angle, value + 0.05, f'{value:.2f}', horizontalalignment='center', size=10, color='black')

        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels[:-1], rotation=45, ha='right')

        plt.title(f"Stacked Radar Chart for Multiple Schemes against AI over {self.trial} trials", size=15, color='black', y=1.1)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
        plt.show()

    def plot_against_all_schemes(self):
        metrics_data = []
        for scheme in self.schemes:
            labels, metrics_values, std_dev_values = self.compute_metrics(scheme, normalization_values=self.normalization_values)
            metrics_data.append((labels, metrics_values, std_dev_values))
        
        self.plot_stacked_radar(metrics_data, self.schemes)

In [5]:
models =  ["main_network_all_features", "main_network_attack_path_exposure", "main_network_exposed_endpoints"]

for model in models:
    radar = RadarPlot(epsilon, start_time, finish_time, mtd_interval, total_nodes, new_network, features, model, trial=trial)
    # Plot against all schemes
    radar.plot_against_all_schemes()



MTD: OSDiversity triggered 0.0s
MTD: CompleteTopologyShuffle triggered 100.3s
MTD: CompleteTopologyShuffle triggered 200.7s
MTD: Interrupted SCAN_PORT at 220.8s!
MTD: CompleteTopologyShuffle triggered 301.0s
MTD: ServiceDiversity triggered 401.2s
MTD: Interrupted SCAN_PORT at 421.2s!
MTD: ServiceDiversity triggered 502.1s


{'record name': 'Vulnerabilities Exploited', 'x': [], 'cumulative exploited vulns': [], 'roa': [], 'impact': [], 'risk': [], 'complexity': [], 'total had os dependency': 0, 'total were dependent on another vuln': 0}
[0.         5.         0.72879819 0.         0.         0.
 0.        ]
{'record name': 'Vulnerabilities Exploited', 'x': [], 'cumulative exploited vulns': [], 'roa': [], 'impact': [], 'risk': [], 'complexity': [], 'total had os dependency': 0, 'total were dependent on another vuln': 0}
[0.         5.         0.79961735 0.         0.         0.
 0.        ]
{'record name': 'Vulnerabilities Exploited', 'x': [], 'cumulative exploited vulns': [], 'roa': [], 'impact': [], 'risk': [], 'complexity': [], 'total had os dependency': 0, 'total were dependent on another vuln': 0}
[0.         5.         0.79961735 0.         0.         0.
 0.        ]
{'record name': 'Vulnerabilities Exploited', 'x': [278.0026193516724], 'cumulative exploited vulns': [1], 'roa': [56.53191849467427], 'i

MTD: ServiceDiversity triggered 602.5s
MTD: CompleteTopologyShuffle triggered 702.7s
MTD: CompleteTopologyShuffle triggered 802.8s
MTD: Interrupted EXPLOIT_VULN at 822.9s!
MTD: CompleteTopologyShuffle triggered 902.9s
MTD: ServiceDiversity triggered 1003.0s
MTD: Interrupted EXPLOIT_VULN at 1023.2s!


{'record name': 'Vulnerabilities Exploited', 'x': [278.0026193516724], 'cumulative exploited vulns': [1], 'roa': [56.53191849467427], 'impact': [9.80258670893613], 'risk': [9.581073376207005], 'complexity': [0.9774025632920758], 'total had os dependency': 1, 'total were dependent on another vuln': 0}
[3.33333333e-03 5.00000000e+00 2.27630952e-01 0.00000000e+00
 5.65319185e+01 0.00000000e+00 9.58107338e+00]
{'record name': 'Vulnerabilities Exploited', 'x': [278.0026193516724], 'cumulative exploited vulns': [1], 'roa': [56.53191849467427], 'impact': [9.80258670893613], 'risk': [9.581073376207005], 'complexity': [0.9774025632920758], 'total had os dependency': 1, 'total were dependent on another vuln': 0}
[3.33333333e-03 5.00000000e+00 2.32169312e-01 0.00000000e+00
 5.65319185e+01 0.00000000e+00 9.58107338e+00]
{'record name': 'Vulnerabilities Exploited', 'x': [278.0026193516724, 767.0318944165143, 767.0318944165143, 767.0318944165143], 'cumulative exploited vulns': [1, 2, 3, 4], 'roa': [

MTD: IPShuffle triggered 1104.4s
MTD: CompleteTopologyShuffle triggered 1205.2s
MTD: Interrupted EXPLOIT_VULN at 1214.8s!
MTD: CompleteTopologyShuffle triggered 1305.6s
MTD: CompleteTopologyShuffle triggered 1405.7s
MTD: Interrupted EXPLOIT_VULN at 1425.8s!
MTD: CompleteTopologyShuffle triggered 1507.2s
MTD: ServiceDiversity triggered 1608.3s


...

MTD: IPShuffle triggered 2110.1s
MTD: OSDiversity triggered 2210.1s
MTD: Interrupted EXPLOIT_VULN at 2220.9s!
MTD: CompleteTopologyShuffle triggered 2311.2s
MTD: ServiceDiversity triggered 2411.4s
MTD: Interrupted EXPLOIT_VULN at 2431.7s!
MTD: ServiceDiversity triggered 2511.6s
MTD: ServiceDiversity triggered 2612.0s


{'record name': 'Vulnerabilities Exploited', 'x': [468.1821510637993, 468.1821510637993, 468.1821510637993, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305], 'cumulative exploited vulns': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'roa': [0.1845219762855362, 1.4311391345077833, 0.7751869793732358, 48.230024974774054, 0.9594682416594114, 0.8720110214791725, 2.988349144517332, 2.1088043042178137, 0.34665003160588537, 19.323205966782513, 3.540303377691395, 23.34929313368981, 13.779531063038137, 4.971763656875096, 4.020518142626846, 1.8705471794401398, 1.7094234774263353, 1.2738241527897476, 0.7570558845255121, 1.29967333237059], 'impact': [1.0463536830244946, 6.28965493167056, 3.35

MTD: OSDiversity triggered 2713.8s
MTD: IPShuffle triggered 2813.8s
MTD: IPShuffle triggered 2914.5s
MTD: Interrupted EXPLOIT_VULN at 2923.9s!
2024-08-21 10:48:57.945015: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-08-21 10:48:57.945037: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-21 10:48:57.945043: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-21 10:48:57.945072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-21 10:48:57.945089: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


{'record name': 'Vulnerabilities Exploited', 'x': [468.1821510637993, 468.1821510637993, 468.1821510637993, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1365.348502201424, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305, 1706.8973488300305], 'cumulative exploited vulns': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'roa': [0.1845219762855362, 1.4311391345077833, 0.7751869793732358, 48.230024974774054, 0.9594682416594114, 0.8720110214791725, 2.988349144517332, 2.1088043042178137, 0.34665003160588537, 19.323205966782513, 3.540303377691395, 23.34929313368981, 13.779531063038137, 4.971763656875096, 4.020518142626846, 1.8705471794401398, 1.7094234774263353, 1.2738241527897476, 0.7570558845255121, 1.29967333237059], 'impact': [1.0463536830244946, 6.28965493167056, 3.35

Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
